In [19]:
#import relevant libraries and info
import os
from together import Together
import csv
import pandas as pd
import keys

client = Together(api_key=keys.TOGETHER_AI_KEY)
MODEL="meta-llama/Llama-3-8b-chat-hf"

Query model

In [7]:
def query_model(model,messages):
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content


In [8]:
messages= [{
    "role": "system",
    "content":"Your job is to judge the interestingness of mathematical statements."},
{
    "role": "user",
    "content": "Judge how interesting the following statement is as a score between 1 and 10, where 1 is utterly boring, and 10 is very interesting.\n\nGiven that (a+f)=a, we have (a+(a+f))=(a+a).\nOutput your answer in the following format:\nScore: {some value}\nReason: {explanation}"
},]
query_model(model=MODEL,messages=messages)

"Score: 2\nReason: This statement is a simple algebraic manipulation, which is not particularly surprising or insightful. It's a straightforward application of the associative property of addition, and the conclusion is not unexpected. The statement lacks any deeper mathematical significance or unexpected twists, making it a relatively uninteresting result."

In [16]:
def extract_score(response):
    score_start = response.find("Score:")
    return response[score_start+7:score_start+8]

In [20]:
df_arr = []
with open('INT\\theorems.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile)  
    # Read each row in the CSV file
    next(csvreader)
    count = 0
    system_message = "Your job is to judge the interestingness of mathematical statements. Please judge how interesting the provided statement is as a score between 1 and 10, where 1 is utterly boring, and 10 is very interesting. Output your answer in the following format:\nScore:{some value}.\nReason: {explanation}."
    for row in csvreader:
        condition = row[1]
        statement = row[2]
        message = [{"role": "system","content":system_message},{"role": "user","content": condition+statement}]
        raw_output = query_model(model=MODEL,messages=messages)
        score = extract_score(raw_output)
        df_arr.append([condition, statement, score, raw_output])
        count+=1
        print(score)
df = pd.DataFrame(df_arr, columns=['condition', 'statement', 'score', 'raw_output'])
df.to_csv('interestingness_judgement.csv')

2
2
2
2


KeyboardInterrupt: 